<a href="https://colab.research.google.com/github/coco12323/first-git/blob/main/Assignment02_solution_ipynp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <span style="color:#0b486b">  FIT5215: Deep Learning (T3, 2020)</span>
***
*Chief Examiner:* Prof **Dinh Phung** | dinh.phung@monash.edu <br/>
*Lecturer:* Dr **Peibo Duan** | peibo.duan@monash.edu <br/>
*Tutor:* Mr **Qing Ye** | qing.ye@monash.edu <br/>

<br/>
Department of Data Science and AI, Faculty of Information Technology, Monash University, Australia <br/>

***

## <span style="color:#0b486b">Assignment 02: Neural Embedding and Sequence Modelling</span>
### Due: <span style="color:red">11:59pm 29 August 2020</span>  (Sunday)

#### <span style="color:red">Important note:</span> This is an **individual** assignment. It contributes **20%** to you final mark. Read the assignment instruction carefully.

## <span style="color:#0b486b">Instructions</span>

This notebook has been prepared for your to complete Assignment 2. The theme of this assignment is about practical machine learning knowledge and skills in deep neural networks, word embedding and text analytics. Some sections have been partially completed to help you get
started. **The total marks for this notebook is 110**.

* Before you start, read the entire notebook carefully once to understand what you need to do. <br><br>
* For each cell marked with **#YOU ARE REQUIRED TO INSERT YOUR CODES IN THIS CELL**, there will be places where you **must** supply your own codes when instructed. <br>

This assignment contains **four** parts:

* Part 1: Questions on downloading and preprocessing data **[10 points]**
* Part 2: Questions on using Word2Vect to transform texts to vectors **[20 points]**
* Part 3: Coding assessment on Text CNN for sequence modeling and neural embedding **[15 points]**
* Part 4: Coding assessment on RNNs for sequence modeling and neural embedding **[65 points]**


**Hint**: This assignment was essentially designed based on the lectures and tutorials sessions covered from Week 5 to 6. You are strongly recommended to go through these contents thoroughly which might help you to complete this assignment.

## <span style="color:#0b486b">What to submit</span>

This assignment is to be completed individually and submitted to Moodle unit site. **By the due date, you are required to submit one  <span style="color:red; font-weight:bold">single zip file, named xxx_assignment02_solution.zip</span> where `xxx` is your student ID, to the corresponding Assignment (Dropbox) in Moodle**. 

***For example, if your student ID is <span style="color:red; font-weight:bold">12356</span>, then gather all of your assignment solution to folder, create a zip file named <span style="color:red; font-weight:bold">123456_assignment02_solution.zip</span> and submit this file.***

Within this zip folder, you **must** submit the following files:
1.	**Assignment02_solution.ipynp**:  this is your Python notebook solution source file.
1.	**Assignment02_output.html**: this is the output of your Python notebook solution *exported* in html format.
1.	Any **extra files or folder** needed to complete your assignment (e.g., images used in your answers).

## <span style="color:#0b486b">Part 1: Download and preprocess the data</span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 10 points]<span></div>

The dataset we use for this assignment is a small [Question Classification](http://cogcomp.org/Data/QA/QC/) dataset which was used in the research paper on the topic of ***Question Classification*** (QC) published by Roth and Xin in 2002 (which has received close to 1000 citations till today). Note that a pdf copy of this paper has been downloaded into this assignment folder (**`Roth_Xin_coling02.pdf`**).

The train set consists of $1,000$ questions belonging to 6 coarse question categories, including:
- abbreviation (ABBR), 
- entity (ENTY), 
- description (DESC), 
- human (HUM), 
- location (LOC) and 
- numeric (NUM).


Preprocessing data is an inital and important step in any machine learning or deep learning projects. The following *DataManager* class helps you to download data and preprocess data for the later steps of a deep learning project. 

In [ ]:
import os
import zipfile
import collections
import numpy as np
import tensorflow as tf
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.style.use('ggplot')

class DataManager:
    def __init__(self, verbose=True, maxlen= 50, random_state=6789):
        self.verbose = verbose
        self.max_sentence_len = 0
        self.str_questions = list()
        self.str_labels = list()
        self.numeral_labels = list()
        self.maxlen = maxlen
        self.numeral_data = list()
        self.random_state = random_state
        self.random = np.random.RandomState(random_state)
        
    @staticmethod
    def maybe_download(dir_name, file_name, url, verbose= True):
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)
        if not os.path.exists(os.path.join(dir_name, file_name)):
            urlretrieve(url + file_name, os.path.join(dir_name, file_name))
        if verbose:
            print("Downloaded successfully {}".format(file_name))
    
    def read_data(self, dir_name, file_names):
        for file_name in file_names:
            file_path= os.path.join(dir_name, file_name)
            self.str_questions= list(); self.str_labels= list()
            with open(file_path, "r", encoding="latin-1") as f:
                for row in f:
                    row_str= row.split(":")
                    label, question= row_str[0], row_str[1]
                    question= question.lower()
                    self.str_labels.append(label)
                    self.str_questions.append(question[0:-1])
                    if self.max_sentence_len < len(self.str_questions[-1]):
                        self.max_sentence_len= len(self.str_questions[-1])
         
        # turns labels into numbers
        le= preprocessing.LabelEncoder()
        le.fit(self.str_labels)
        self.numeral_labels = np.array(le.transform(self.str_labels))
        self.str_classes= le.classes_
        self.num_classes= len(self.str_classes)
        if self.verbose:
            print("\nSample questions... \n")
            print(self.str_questions[0:5])
            print("Labels {}\n\n".format(self.str_classes))
    
    def manipulate_data(self):
        tokenizer = tf.keras.preprocessing.text.Tokenizer()
        tokenizer.fit_on_texts(self.str_questions)
        self.numeral_data = tokenizer.texts_to_sequences(self.str_questions)
        self.numeral_data = tf.keras.preprocessing.sequence.pad_sequences(self.numeral_data, padding='post', truncating= 'post', maxlen= self.maxlen)##填充
        self.word2idx = tokenizer.word_index
        self.word2idx = {k:v for k,v in self.word2idx.items()}
        self.idx2word = {v:k for k,v in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)
    
    def train_valid_split(self, train_ratio=0.9):
        idxs = np.random.permutation(np.arange(len(self.str_questions)))
        train_size = int(train_ratio*len(idxs)) +1
        self.train_str_questions, self.valid_str_questions = self.str_questions[0:train_size], self.str_questions[train_size:]#str句子
        self.train_numeral_data, self.valid_numeral_data = self.numeral_data[0:train_size], self.numeral_data[train_size:]#序列化的句子
        self.train_numeral_labels, self.valid_numeral_labels = self.numeral_labels[0:train_size], self.numeral_labels[train_size:]#label
        self.tf_train_set = tf.data.Dataset.from_tensor_slices((self.train_numeral_data, self.train_numeral_labels))
        self.tf_valid_set = tf.data.Dataset.from_tensor_slices((self.valid_numeral_data, self.valid_numeral_labels))

In [ ]:
print('Loading data...')
DataManager.maybe_download("Data", "train_1000.label", "http://cogcomp.org/Data/QA/QC/")
DataManager.maybe_download("Data", "TREC_10.label", "http://cogcomp.org/Data/QA/QC/")

dm = DataManager(maxlen=100)
dm.read_data("Data/", ["train_1000.label", "TREC_10.label"])   # read data

Loading data...
Downloaded successfully train_1000.label
Downloaded successfully TREC_10.label

Sample questions... 

['dist how far is it from denver to aspen ?', 'city what county is modesto , california in ?', 'desc who was galileo ?', 'def what is an atom ?', 'date when did hawaii become a state ?']
Labels ['ABBR' 'DESC' 'ENTY' 'HUM' 'LOC' 'NUM']




In [ ]:
dm.manipulate_data()
dm.train_valid_split(train_ratio=0.8)

You now have a data manager, named *dm* containing the training and validiation sets in both text and numeric forms. Your task is to play around and read this code to figure out the meanings of some important attributes that will be used in the next parts.

#### <span style="color:red">**Question 1.1**</span> 
**What is the purpose of `self.train_str_questions` and `self.train_numeral_labels`? Write your code to print out the first five questions with labels in the training set.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

#Your answer here



self.train_str_questions are text questions in string format

self.train_numeral_labels are labels in numeric form

In [ ]:
# the first five questions with labels in the training set.
my_train_set = tf.data.Dataset.from_tensor_slices((dm.train_str_questions, dm.train_numeral_labels))
list(my_train_set.as_numpy_iterator())[:5]

[(b'dist how far is it from denver to aspen ?', 5),
 (b'city what county is modesto , california in ?', 4),
 (b'desc who was galileo ?', 3),
 (b'def what is an atom ?', 1),
 (b'date when did hawaii become a state ?', 5)]

#### <span style="color:red">**Question 1.2**</span> 
**What is the purpose of `self.train_numeral_data`? Write your code to print out the first five questions in the numeric format with labels in the training set.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

#Your answer here



self.train_numeral_data is the text question in numerical form

In [ ]:
#Your code here
list(dm.tf_train_set.as_numpy_iterator())[:5]#array[numeral,labels]

[(array([ 27,  14,  73,   3, 115,  39, 305,  20, 306,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32), 5),
 (array([ 18,   1, 153,   3, 307, 308,   6,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0, 

#### <span style="color:red">**Question 1.3**</span> 
**What is the purpose of two dictionaries: `self.word2idx` and `self.idx2word`? Write your code to print out the first five key-value pairs of those dictionaries.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

#Your answer here



self.word2idx is the sequence number corresponding to the word in this question

self.idx2word is the word in the question corresponding to the serial number

In [ ]:
#the first five key-value pairs of those dictionaries
for i,(k,v) in enumerate(dm.idx2word.items()):
    if i in range(0,5):
        print({k:v},end=""+"\n")
    

{1: 'what'}
{2: 'the'}
{3: 'is'}
{4: 'def'}
{5: 'of'}


#### <span style="color:red">**Question 1.4**</span> 
**What is the purpose of `self.tf_train_set`? Write your code to print out the first five items of `self.tf_train_set`.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

#Your answer here


self.tf_train_set is the populated train_numeral_data and self.train_numeral_labels

Digitized text questions and labels for the saved training set

Save in the format of array[train_numeral_data, self.train_numeral_labels]

In [ ]:
# print out the first five items of self.tf_train_set
list(dm.tf_train_set.as_numpy_iterator())[:5]#array[numeral,labels]

[(array([ 27,  14,  73,   3, 115,  39, 305,  20, 306,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32), 5),
 (array([ 18,   1, 153,   3, 307, 308,   6,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0, 

#### <span style="color:red">**Question 1.5**</span> 
**What is the purpose of `self.tf_valid_set`? Write your code to print out the first five items of `self.tf_valid_set`.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

#Your answer here


self.tf_valid_set is the populated valid_numeral_data and self.valid_numeral_labels

Digitized text questions and labels for the saved validation set

Save in the format of array[valid_numeral_data, self.valid_numeral_labels]

In [ ]:
# print out the first five items of self.tf_valid_set.
list(dm.tf_valid_set.as_numpy_iterator())[:5]#array[numeral,labels]

[(array([ 32,   1,   3,   2, 921, 922, 923, 924,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32), 2),
 (array([ 11,  17,   3, 925,  15,  69,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0, 

## <span style="color:#0b486b">Part 2: Using Word2Vect to transform texts to vectors </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 20 points]<span></div>

In this part, you will be assessed on how to use a pretrained Word2Vect model for realizing a machine learning task. Basically, you will use this pretrained Word2Vect to transform the questions in the above dataset stored in the *data manager object dm* to numeric form for training a Support Vector Machine in sckit-learn.  

In [ ]:
import gensim.downloader as api
from gensim.models import Word2Vec
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

#### <span style="color:red">**Question 2.1**</span> 
**Write code to download the pretrained model *glove-wiki-gigaword-100*. Note that this model transforms a word in its dictionary to a $100$ dimensional vector.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [ ]:
import gensim.downloader as api
info = api.info()  # show info about available models/datasets
word2vect = api.load("glove-wiki-gigaword-100")#Insert your code here

[==================================================] 100.0% 128.1/128.1MB downloaded


#### <span style="color:red">**Question 2.2**</span> 

**Write code for the function *get_word_vector(word, model)* used to transform a word to a vector using the pretrained Word2Vect model *model*. Note that for a word not in the vocabulary of our *word2vect*, you need to return a vector $0$ with 100 dimensions.**

<div style="text-align: right"><span style="color:red">[3 points]</span></div> 

In [ ]:
def get_word_vector(word, model):
    try:
        vector = word2vect[word]#Insert your code here
    except: #word not in the vocabulary
        vector = np.zeros((100,), dtype="float32")#Insert your code here
    return vector

#### <span style="color:red">**Question 2.3**</span> 

**Write the code for the function *get_sentence_vector(sentence, important_score=None, model= None)*. Note that this function will transform a sentence to a 100-dimensional vector using the pretrained model *model*. In addition, the list *important_score* which has the same length as the *sentence* specifies the important scores of the words in the sentence. In your code, you first need to apply *softmax* function over *important_score* to obtain the important weight *important_weight* which forms a probability over the words of the sentence. Furthermore, the final vector of the sentence will be weighted sum of the individual vectors for words and the weights in *important_weight*.**
- $final\_vector= important\_weight[1]\times v[1] + important\_weight[2]\times v[2] + ...+ important\_weight[L]\times v[L]$ where $L$ is the length of the sentence and $v[i]$ is the vector of the word $i-th$ in this sentence.

**Note that if *important_score=None* is set by default, your function should return the average of all representation vectors corresponding to set *important_score=[1,1,...,1]*.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div> 

In [ ]:
from scipy.special import softmax
def get_sentence_vector(sentence, important_score=None, model= None):
    mysentence = sentence.split(' ')
    L = len(mysentence)
    final_vector= np.zeros((100,), dtype="float32")
    if important_score is None:
        important_score = np.ones((L,), dtype="float32")
    else:
        important_score = tf.nn.softmax(important_score)
    
    for i in range (L):
        vector_i = get_word_vector(mysentence[i], model)
        final_vector = final_vector + important_score[i]*vector_i
    
    return final_vector
    #Insert your code here

#### <span style="color:red">**Question 2.4**</span> 

**Write code to transform the training questions in *dm.train_str_questions* to feature vectors. Note that after running the following cell, you must have $X\_train$ which is an numpy array of the feature vectors and $y\_train$ which is an array of numeric labels (*Hint: dm.train_numeral_labels*). You can add more lines to the following cell if necessary. In addition, you should decide the *important_score* by yourself. For example, you might reckon that the 1st score is 1, the 2nd score is decayed by 0.9, the 3rd is decayed by 0.9, and so on.**

<div style="text-align: right"><span style="color:red">[2.5 points]</span></div> 

In [ ]:
print("Transform training set to feature vectors...")
my_important_score = np.logspace(0,100,100,base=0.9)
final_vector=list()
for i in dm.train_str_questions:
    final_vector.append( get_sentence_vector(i, important_score = None, model= word2vect) )
X_train = np.array(final_vector,dtype="float32")
y_train = dm.train_numeral_labels.reshape(-1,1)

Transform training set to feature vectors...


#### <span style="color:red">**Question 2.5**</span> 

**Write code to transform the training questions in *dm.valid_str_questions* to feature vectors. Note that after running the following cell, you must have $X\_valid$ which is an numpy array of the feature vectors and $y\_valid$ which is an array of numeric labels (*Hint: dm.valid_numeral_labels*). You can add more lines to the following cell if necessary. In addition, you should decide the *important_score* by yourself. For example, you might reckon that the 1st score is 1, the 2nd score is decayed by 0.9, the 3rd is decayed by 0.9, and so on.**

<div style="text-align: right"><span style="color:red">[2.5 points]</span></div> 

In [ ]:
print("Transform valid set to feature vectors...")
final_vector=list()
for i in dm.valid_str_questions:
    final_vector.append( get_sentence_vector(i, important_score = my_important_score, model= word2vect) )
X_valid = np.array(final_vector,dtype="float32")
y_valid = dm.valid_numeral_labels.reshape(-1,1)

Transform valid set to feature vectors...


#### <span style="color:red">**Question 2.6**</span> 

**It is now to use *MinMaxScaler(feature_range=(-1,1))* in sckit-learn to scale both training and valid sets to the range $(-1,1)$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler(feature_range=(-1,1))
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
# scaler.fit(X_train)
# X_train = #Insert your code here
# scaler.fit(X_valid)
# X_valid = #Insert your code here

In [ ]:
X_train.shape

(401, 100)

In [ ]:
X_valid.shape

(99, 100)

#### <span style="color:red">**Question 2.7**</span> 

**Declare a support vector machine (the class *SVC*  in sckit-learn) with RBF kernel, $C=1$, $gamma= 2^{-3}$ and fit on the training set.**

<div style="text-align: right"><span style="color:red">[3 points]</span></div> 

In [ ]:
from sklearn import svm
mysvm = svm.SVC(C=1, kernel='rbf', gamma=1/8, decision_function_shape='ovr')
mysvm.fit(X_train , y_train.ravel())
#Insert your code for fitting svm on X_train

SVC(C=1, gamma=0.125)

#### <span style="color:red">**Question 2.8**</span> 

**Finally, we use the trained *svm* to evaluate on the valid set $X\_valid$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div> 

In [ ]:
y_valid_pred= mysvm.predict(X_valid)
acc = accuracy_score(y_valid,y_valid_pred)
print(acc)

0.8282828282828283


## <span style="color:#0b486b">Part 3: Text CNN for sequence modeling and neural embedding </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 15 points]<span></div>

#### <span style="color:red">**Question 3.1**</span> 

**In what follows, you are required to complete the code for Text CNN for sentence classification. The paper of Text CNN can be found at this [link](https://www.aclweb.org/anthology/D14-1181.pdf). Here is the description of the Text CNN you need to construct.**
- There are three attributes (properties or instance variables): *embed_size, state_size, data_manager*.
  - `embed_size`: the dimension of the vector space for which the words are embedded to using the embedding matrix.
  - `state_size`: the number of filters used in *Conv1D* (reference [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv1D)).
  - `data_manager`: the data manager to store information of the dataset.
- The detail of the computational process is as follows:
  - Given input $x$, we embed $x$ using the embedding matrix to obtain an $3D$ tensor $[batch\_size \times vocab\_size \times embed\_size]$ as $h$.
  - We feed $h$ to three Convd 1D layers, each of which has $state\_size$ filters, padding=same, activation= relu, and $kernel\_size= 3, 5, 7$ respectively to obtain $h1, h2, h3$. Note that each $h1, h2, h3$ is a 3D tensor with the shape $[batch\_size \times output\_size \times state\_size]$.
  - We then apply *GlobalMaxPool1D()* (reference [here](https://www.tensorflow.org/api_docs/python/tf/keras/layers/GlobalMaxPool1D)) over $h1, h2, h3$ to obtain 2D tensors stored in $h1, h2, h3$ again.
  - We then concatenate three 2D tensors $h1, h2, h3$ to obtain $h$. Note that you need to specify the axis to concatenate.
  - We finally build up one dense layer on the top of $h$ for classification.
  
  <div style="text-align: right"><span style="color:red">[10 points]</span></div>
  

In [ ]:
from keras.layers import Conv1D, MaxPool1D, Dense, Flatten, Concatenate, Embedding,Input
from keras.models import Model

In [ ]:
from keras.layers.pooling import GlobalMaxPool1D
class TextCNN:
    def __init__(self, embed_size= 128, state_size=16, data_manager=None):
        self.data_manager = data_manager
        self.embed_size = embed_size
        self.state_size = state_size
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])

        h = tf.keras.layers.Embedding(self.data_manager.vocab_size +1, self.embed_size)(x)
        
        h1 = Conv1D(filters=self.state_size, padding='same', activation= 'relu', kernel_size=3)(h)
        h2 = Conv1D(filters=self.state_size, padding='same', activation= 'relu', kernel_size=5)(h)
        h3 = Conv1D(filters=self.state_size, padding='same', activation= 'relu', kernel_size=7)(h)
        
        h1 = GlobalMaxPool1D(data_format='channels_last')(h1)
        h2 = GlobalMaxPool1D(data_format='channels_last')(h2)
        h3 = GlobalMaxPool1D(data_format='channels_last')(h3)
        
        h = Concatenate(axis=1)([h1,h2,h3])#Insert your code here
        
        h = tf.keras.layers.Dense(self.data_manager.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h) 
    
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)


#### <span style="color:red">**Question 3.2**</span> 
**Here is the code to test TextCNN above. You can observe that TextCNN outperforms the traditional approach SVM + Word2Vect for this task. The reason is that TextCNN enables us to automatically learn the feature that fits to the task. This makes deep learning different from hand-crafted feature approaches. Complete the code to test the model. Note that when compiling the model, you can use the Adam optimizer.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [ ]:
text_cnn = TextCNN(data_manager=dm)
text_cnn.build()
text_cnn.compile_model(optimizer="adam", loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),metrics=["accuracy"])
#Insert code here to compile the model
text_cnn.model.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))
#Insert code here to train the model on 20 epochs

Epoch 1/20
7/7 [==============================] - 2s 112ms/step - loss: 1.7824 - accuracy: 0.2145 - val_loss: 1.7284 - val_accuracy: 0.3838
Epoch 2/20
7/7 [==============================] - 1s 84ms/step - loss: 1.6479 - accuracy: 0.4863 - val_loss: 1.6523 - val_accuracy: 0.4040
Epoch 3/20
7/7 [==============================] - 1s 80ms/step - loss: 1.5327 - accuracy: 0.5112 - val_loss: 1.5645 - val_accuracy: 0.4141
Epoch 4/20
7/7 [==============================] - 1s 82ms/step - loss: 1.4025 - accuracy: 0.5711 - val_loss: 1.4524 - val_accuracy: 0.4343
Epoch 5/20
7/7 [==============================] - 1s 81ms/step - loss: 1.2526 - accuracy: 0.6334 - val_loss: 1.3254 - val_accuracy: 0.5051
Epoch 6/20
7/7 [==============================] - 1s 80ms/step - loss: 1.0885 - accuracy: 0.6958 - val_loss: 1.1762 - val_accuracy: 0.5455
Epoch 7/20
7/7 [==============================] - 1s 82ms/step - loss: 0.9134 - accuracy: 0.8554 - val_loss: 1.0173 - val_accuracy: 0.6667
Epoch 8/20
7/7 [==========

## <span style="color:#0b486b">Part 4: RNNs for sequence modeling and neural embedding </span>

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 65 points]<span></div>

### <span style="color:#0b486b">4.1. One-directional RNNs for sequence modeling and neural embedding </span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 20 points]<span></div>

#### <span style="color:red">**Question 4.1.1**</span> 
**In this part, you need to construct an RNN to learn from the dataset of interest. Basically, you are required first to construct the class UniRNN (Uni-directional RNN) with the following requirements:**
- Attribute `data_manager (self.data_manager)`: specifies the data manager used to store data for the model.
- Attribute `cell_type (self.cell_type)`: can receive three values including `basic_rnn`, `gru`, and `lstm` which specifies the memory cells formed a hidden layer.
- `state_sizes (self.state_sizes)` indicates the list of the hidden sizes from the second hidden layers of memory cells. For example, $embed\_size =128$ and $state\_sizes = [64, 64]$ means that you have three hidden layers in your network with hidden sizes of $128, 64$ and $64$ respectively.

**Note that when declaring an embedding layer for the network, you need to set *mask_zero=True* so that the padding zeros in the sentences will be masked and ignored. This helps to have variable length RNNs. For more detail, you can refer to this [link](https://www.tensorflow.org/guide/keras/masking_and_padding).**

<div style="text-align: right"><span style="color:red">[10 points]</span></div>

In [ ]:
from keras.layers import GRU, LSTM, RNN, Bidirectional

In [ ]:
class UniRNN:
    def __init__(self, cell_type= 'gru', embed_size= 128, state_sizes= [128, 64], data_manager= None):
        self.cell_type = cell_type
        self.state_sizes = state_sizes
        self.embed_size = embed_size
        self.data_manager = data_manager
        self.vocab_size = self.data_manager.vocab_size +1 
        
    #return the correspoding memory cell
    @staticmethod
    def get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh'):
        if cell_type== 'gru':
            return GRU(state_size, return_sequences = return_sequences, activation = activation)#Insert your code here
        elif cell_type== 'lstm':
            return LSTM(state_size, return_sequences = return_sequences, activation = activation)#Insert your code here
        else:
            return RNN(state_size, return_sequences = return_sequences, activation = activation)#Insert your code here
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = Embedding(self.vocab_size, self.embed_size,mask_zero=True)(x)#Insert your code here
        num_layers = len(self.state_sizes)#Insert your code here
        for i in range(num_layers):
            if (i ==  num_layers-1):
                layer = self.get_layer(state_size=self.state_sizes[i])
            else:
                layer = self.get_layer(state_size=self.state_sizes[i],return_sequences= True)
            h = layer(h) #Insert your code here, you can insert more lines if necessary
        h = tf.keras.layers.Dense(dm.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)
   
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


#### <span style="color:red">**Question 4.1.2**</span> 
**Run with basic RNN ('basic_rnn') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [ ]:
uni_rnn = UniRNN(cell_type= 'basic_rnn', embed_size= 128, state_sizes= [128, 128],data_manager=dm)#Insert your code here
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 14s 912ms/step - loss: 1.6249 - accuracy: 0.4015 - val_loss: 1.3798 - val_accuracy: 0.3939
Epoch 2/20
7/7 [==============================] - 3s 457ms/step - loss: 1.0835 - accuracy: 0.5436 - val_loss: 1.2462 - val_accuracy: 0.4646
Epoch 3/20
7/7 [==============================] - 3s 455ms/step - loss: 0.8501 - accuracy: 0.6359 - val_loss: 1.1352 - val_accuracy: 0.4545
Epoch 4/20
7/7 [==============================] - 4s 555ms/step - loss: 0.7123 - accuracy: 0.7406 - val_loss: 0.9301 - val_accuracy: 0.6162
Epoch 5/20
7/7 [==============================] - 3s 456ms/step - loss: 0.4899 - accuracy: 0.8603 - val_loss: 0.9394 - val_accuracy: 0.6162
Epoch 6/20
7/7 [==============================] - 3s 460ms/step - loss: 0.3236 - accuracy: 0.9227 - val_loss: 0.9735 - val_accuracy: 0.6162
Epoch 7/20
7/7 [==============================] - 4s 562ms/step - loss: 0.2150 - accuracy: 0.9651 - val_loss: 0.9789 - val_accuracy: 0.6364
Epoch 8/20
7/7 [===

#### <span style="color:red">**Question 4.1.3**</span> 
**Run with GRU ('gru') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [ ]:
uni_rnn = UniRNN(cell_type= 'gru', embed_size= 128, state_sizes= [128, 128],data_manager=dm) #Insert your code here
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 14s 891ms/step - loss: 1.6514 - accuracy: 0.3691 - val_loss: 1.4606 - val_accuracy: 0.3636
Epoch 2/20
7/7 [==============================] - 3s 462ms/step - loss: 1.0850 - accuracy: 0.5736 - val_loss: 1.2296 - val_accuracy: 0.4949
Epoch 3/20
7/7 [==============================] - 3s 447ms/step - loss: 0.7497 - accuracy: 0.6908 - val_loss: 1.0291 - val_accuracy: 0.5253
Epoch 4/20
7/7 [==============================] - 3s 449ms/step - loss: 0.4824 - accuracy: 0.8404 - val_loss: 0.9286 - val_accuracy: 0.6465
Epoch 5/20
7/7 [==============================] - 3s 453ms/step - loss: 0.3171 - accuracy: 0.9027 - val_loss: 0.5813 - val_accuracy: 0.7677
Epoch 6/20
7/7 [==============================] - 3s 454ms/step - loss: 0.2061 - accuracy: 0.9451 - val_loss: 0.8587 - val_accuracy: 0.7677
Epoch 7/20
7/7 [==============================] - 3s 454ms/step - loss: 0.0974 - accuracy: 0.9925 - val_loss: 0.4569 - val_accuracy: 0.8283
Epoch 8/20
7/7 [===

#### <span style="color:red">**Question 4.1.4**</span> 
**Run with LSTM ('lstm') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [ ]:
uni_rnn = UniRNN(cell_type= 'lstm', embed_size= 128, state_sizes= [128, 128],data_manager=dm)#Insert your code here
uni_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
uni_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
uni_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 15s 1s/step - loss: 1.6542 - accuracy: 0.3791 - val_loss: 1.4446 - val_accuracy: 0.3939
Epoch 2/20
7/7 [==============================] - 3s 454ms/step - loss: 1.0775 - accuracy: 0.5711 - val_loss: 1.1594 - val_accuracy: 0.4949
Epoch 3/20
7/7 [==============================] - 3s 454ms/step - loss: 0.7064 - accuracy: 0.6983 - val_loss: 0.9575 - val_accuracy: 0.5758
Epoch 4/20
7/7 [==============================] - 3s 452ms/step - loss: 0.4259 - accuracy: 0.8454 - val_loss: 0.8463 - val_accuracy: 0.6768
Epoch 5/20
7/7 [==============================] - 3s 452ms/step - loss: 0.3374 - accuracy: 0.8878 - val_loss: 0.5136 - val_accuracy: 0.7879
Epoch 6/20
7/7 [==============================] - 3s 451ms/step - loss: 0.1379 - accuracy: 0.9825 - val_loss: 0.6216 - val_accuracy: 0.7980
Epoch 7/20
7/7 [==============================] - 3s 453ms/step - loss: 0.0749 - accuracy: 0.9875 - val_loss: 0.5014 - val_accuracy: 0.8485
Epoch 8/20
7/7 [======

#### <span style="color:red">**Question 4.1.5**</span> 
**Give your own comments about the performance of three memory cells for the dataset of interest as well as what happening during the training process of each cell. Note that there are not right or wrong comments and your comments rely on the status of your training. In addition, some comments and hypothesized assessments of what and why are occurring are useful to obtain the highest score for this question.**

<div style="text-align: right"><span style="color:red">[4 points]</span></div>

#Your comments here


All three models get a good score of accuracy= 1.0000 in the training set

basic_rnn gets accuracy= 1.0000 at 17 epochs
gru gets accuracy= 1.0000 at 14 epochs
lstm gets accuracy= 1.0000 at epoch 13

On the validation set basic_rnn: val_accuracy=0.8788, gru: val_accuracy=0.8788, lstm: val_accuracy: 0.8990
The final result obtained by lstm is better than basic_rnn, gru
LSTM is an excellent variant model of RNN. It inherits the characteristics of the RNN model and converges faster.

### <span style="color:#0b486b">4.2. Bi-directional RNNs for sequence modeling and neural embedding </span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 20 points]<span></div>

#### <span style="color:red">**Question 4.2.1**</span> 
**In what follow, you will investigate BiRNN. The task is similar to Part 4.1 but you need to write the code for an BiRNN. Note that the function *get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh')* has to return the hidden layer with bidirectional memory cells (e.g., Basic RNN, GRU, and LSTM cells).**

**Complete the code of the class *BiRNN*. Note that for the embedding layer you need to set *mask_zero=True*.**

<div style="text-align: right"><span style="color:red">[10 points]</span></div>

In [ ]:
class BiRNN:
    def __init__(self, cell_type= 'gru', embed_size= 128, state_sizes= [128, 64], data_manager= None):
        self.cell_type = cell_type
        self.state_sizes = state_sizes
        self.embed_size = embed_size
        self.data_manager = data_manager
        self.vocab_size = self.data_manager.vocab_size +1
        
    @staticmethod
    def get_layer(cell_type= 'gru', state_size= 128, return_sequences= False, activation = 'tanh'):
        if cell_type=='gru':
            return Bidirectional(GRU(state_size,return_sequences= return_sequences, activation = activation))#Insert your code here
        elif cell_type== 'lstm':
            return Bidirectional(LSTM(state_size,return_sequences= return_sequences, activation = activation))#Insert your code here
        else:
            return Bidirectional(RNN(state_size,return_sequences= return_sequences, activation = activation))#Insert your code here
    
    def build(self):
        x = tf.keras.layers.Input(shape=[None])
        h = Embedding(self.vocab_size, self.embed_size,mask_zero=True)(x)#Insert your code here
        num_layers = len(self.state_sizes)#Insert your code here
        for i in range(num_layers):
            if (i ==  num_layers-1):
                layer = self.get_layer(state_size=self.state_sizes[i])
            else:
                layer = self.get_layer(state_size=self.state_sizes[i],return_sequences= True)
            h = layer(h)#Insert your code here, add more lines if necessary
        h = tf.keras.layers.Dense(dm.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)
        
    
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


#### <span style="color:red">**Question 4.2.2**</span> 
**Run BiRNN for basic RNN ('basic_rnn') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [ ]:
bi_rnn = BiRNN(cell_type= 'bi_rnn', embed_size= 128, state_sizes= [128, 128],data_manager=dm)#Insert your code here
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 30s 2s/step - loss: 1.5684 - accuracy: 0.4364 - val_loss: 1.2475 - val_accuracy: 0.4242
Epoch 2/20
7/7 [==============================] - 9s 1s/step - loss: 0.8800 - accuracy: 0.6733 - val_loss: 1.0802 - val_accuracy: 0.5253
Epoch 3/20
7/7 [==============================] - 8s 1s/step - loss: 0.5490 - accuracy: 0.8055 - val_loss: 0.6421 - val_accuracy: 0.7071
Epoch 4/20
7/7 [==============================] - 8s 1s/step - loss: 0.2822 - accuracy: 0.9277 - val_loss: 0.5116 - val_accuracy: 0.8182
Epoch 5/20
7/7 [==============================] - 8s 1s/step - loss: 0.1207 - accuracy: 0.9800 - val_loss: 0.3898 - val_accuracy: 0.8687
Epoch 6/20
7/7 [==============================] - 8s 1s/step - loss: 0.0586 - accuracy: 0.9825 - val_loss: 0.3609 - val_accuracy: 0.8586
Epoch 7/20
7/7 [==============================] - 8s 1s/step - loss: 0.0324 - accuracy: 0.9900 - val_loss: 0.2139 - val_accuracy: 0.9495
Epoch 8/20
7/7 [========================

#### <span style="color:red">**Question 4.2.3**</span> 
**Run BiRNN for GRU ('gru') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [ ]:
bi_rnn = BiRNN(cell_type= 'gru', embed_size= 128, state_sizes= [128, 128],data_manager=dm)#Insert your code here
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 30s 2s/step - loss: 1.5729 - accuracy: 0.4190 - val_loss: 1.2782 - val_accuracy: 0.4040
Epoch 2/20
7/7 [==============================] - 8s 1s/step - loss: 0.9297 - accuracy: 0.6459 - val_loss: 0.9584 - val_accuracy: 0.5253
Epoch 3/20
7/7 [==============================] - 8s 1s/step - loss: 0.5009 - accuracy: 0.8429 - val_loss: 0.8392 - val_accuracy: 0.6869
Epoch 4/20
7/7 [==============================] - 8s 1s/step - loss: 0.2430 - accuracy: 0.9352 - val_loss: 0.7231 - val_accuracy: 0.7273
Epoch 5/20
7/7 [==============================] - 14s 2s/step - loss: 0.1000 - accuracy: 0.9776 - val_loss: 0.6029 - val_accuracy: 0.7980
Epoch 6/20
7/7 [==============================] - 8s 1s/step - loss: 0.0490 - accuracy: 0.9825 - val_loss: 0.3835 - val_accuracy: 0.8687
Epoch 7/20
7/7 [==============================] - 8s 1s/step - loss: 0.0243 - accuracy: 0.9925 - val_loss: 0.4173 - val_accuracy: 0.8990
Epoch 8/20
7/7 [=======================

#### <span style="color:red">**Question 4.2.4**</span> 
**Run BiRNN for LSTM ('lstm') cell with $embed\_size= 128, state\_sizes= [128, 128], data\_manager= dm$.**

<div style="text-align: right"><span style="color:red">[2 points]</span></div>

In [ ]:
bi_rnn = BiRNN(cell_type= 'lstm', embed_size= 128, state_sizes= [128, 128],data_manager=dm)#Insert your code here
bi_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
bi_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
bi_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 33s 2s/step - loss: 1.5703 - accuracy: 0.4464 - val_loss: 1.3448 - val_accuracy: 0.4242
Epoch 2/20
7/7 [==============================] - 7s 1s/step - loss: 0.9018 - accuracy: 0.6534 - val_loss: 0.9249 - val_accuracy: 0.5455
Epoch 3/20
7/7 [==============================] - 7s 1s/step - loss: 0.5214 - accuracy: 0.8354 - val_loss: 0.7491 - val_accuracy: 0.6869
Epoch 4/20
7/7 [==============================] - 7s 1s/step - loss: 0.2697 - accuracy: 0.9277 - val_loss: 0.5973 - val_accuracy: 0.7879
Epoch 5/20
7/7 [==============================] - 7s 1s/step - loss: 0.2272 - accuracy: 0.9202 - val_loss: 0.9126 - val_accuracy: 0.6869
Epoch 6/20
7/7 [==============================] - 7s 1s/step - loss: 0.0943 - accuracy: 0.9676 - val_loss: 0.5625 - val_accuracy: 0.8081
Epoch 7/20
7/7 [==============================] - 12s 2s/step - loss: 0.0501 - accuracy: 0.9875 - val_loss: 0.3550 - val_accuracy: 0.8889
Epoch 8/20
7/7 [=======================

#### <span style="color:red">**Question 4.2.5**</span> 

**Give your own comments about the performance of three memory cells for the dataset of interest as well as comparing BiRNN to UniRNN in Part 1.**

<div style="text-align: right"><span style="color:red">[4 points]</span></div>

#Your comments here


#Your comments here
Similar to UniRNN, all three models got accuracy= 1.0000 after BiRNN training



basic_rnn gets accuracy= 1.0000 at 8 epochs
gru gets accuracy= 1.0000 at 8 epochs
lstm gets accuracy= 1.0000 at the 9th epoch

On validation set basic_rnn: val_accuracy=0.9293, gru: val_accuracy= 0.8990, lstm: val_accuracy= 0.8788

The result of basic_rnn in BiRNN is better than lstm, gru
lstm is not as good as basic_rnn, gru in BiRNN, slower convergence, lowest val_accuracy

### <span style="color:#0b486b">4.3. RNNs with various types, cells, and fine-tuning embedding matrix for sequence modeling and neural embedding </span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 20 points]<span></div>

#### <span style="color:red">**Question 4.3.1**</span> 

**In what follows, you are required to combine the code in Part 1 and Part 2 to gain a general RNN which can be either Uni-directional RNN or Bi-directional RNN and the embedding matrix can be initialized using a pretrained Word2Vect.**

**Below are the descriptions of the attributes of the class *RNN*:**
- `run_mode (self.run_mode)` has three values (scratch, init-only, and init-fine-tune).
  - `scratch` means training the embedding matrix from scratch.
  - `init-only` means only initialzing the embedding matrix with a pretrained Word2Vect but not further doing fine-tuning that matrix.
  - `init-fine-tune` means both initialzing the embedding matrix with a pretrained Word2Vect and further doing fine-tuning that matrix.
- `network_type (self.network_type)` has two values (uni-directional and bi-directional) which correspond to either Uni-directional RNN or Bi-directional RNN.
- `cell_type (self.cell_type)` has three values (simple-rnn, gru, and lstm) which specify the memory cell used in the network.
- `embed_model (self.embed_model)` specifes the pretrained Word2Vect model used.
-  `embed_size (self.embed_size)` specifes the embedding size. Note that when run_mode is either init-only' or 'init-fine-tune', this embedding size is extracted from embed_model for dimension compatability.
- `state_sizes (self.state_sizes)` indicates the list of the hidden sizes from the second hidden layers of memory cells. For example, $embed\_size =128$ and $state\_sizes = [64, 64]$ means that you have three hidden layers in your network with hidden sizes of $128, 64$ and $64$ respectively.

**Complete the code of the class *RNN*.**

<div style="text-align: right"><span style="color:red">[10 points]</span></div>

In [1]:
class RNN:
    def __init__(self, run_mode = 'scratch', cell_type= 'gru', network_type = 'uni-directional', embed_model= 'glove-wiki-gigaword-100', 
                 embed_size= 128, state_sizes = [64, 64], data_manager = None):
        self.run_mode = run_mode
        self.data_manager = data_manager
        self.cell_type = cell_type
        self.network_type = network_type
        self.state_sizes = state_sizes
        self.embed_model = embed_model
        self.embed_size = embed_size
        if self.run_mode != 'scratch':
            self.embed_size = int(self.embed_model.split("-")[-1])
        self.data_manager = data_manager
        self.vocab_size = dm.vocab_size +1
        self.word2idx = dm.word2idx
        self.word2vect = None
        self.embed_matrix = np.zeros(shape= [self.vocab_size, self.embed_size])
    
    def build_embedding_matrix(self):
        self.word2vect = api.load(self.embed_model)#Insert your code here
        for word, idx in self.word2idx.items():
            try:
                self.embed_matrix[idx] = self.word2vect.word_vec(word)
            except KeyError:
                pass     
        #Insert your code here
    
    @staticmethod
    def get_layer(cell_type= 'gru', network_type= 'uni-directional', hidden_size= 128, return_sequences= False, activation = 'tanh'):
        if (network_type == 'uni-directional'):
            if cell_type== 'gru':
                return GRU(hidden_size, return_sequences = return_sequences, activation = activation)#Insert your code here
            elif cell_type== 'lstm':
                return LSTM(hidden_size, return_sequences = return_sequences, activation = activation)#Insert your code here
            else:
                return RNN(hidden_size, return_sequences = return_sequences, activation = activation)#Insert your code here
    

        else:
            if cell_type=='gru':
                return Bidirectional(GRU(hidden_size,return_sequences= return_sequences, activation = activation))#Insert your code here
            elif cell_type== 'lstm':
                return Bidirectional(LSTM(hidden_size,return_sequences= return_sequences, activation = activation))#Insert your code here
            else:
                return Bidirectional(RNN(hidden_size,return_sequences= return_sequences, activation = activation))#Insert your code here
    #Insert your code here
    
    def build(self):
        x = Input(shape=[None])
        if self.run_mode == "scratch":
            self.embedding_layer = Embedding(self.vocab_size, self.embed_size, trainable= True,mask_zero=True)
        elif self.run_mode == "init-only":
            self.build_embedding_matrix()
            self.embedding_layer = Embedding(self.vocab_size, self.embed_size, weights= [self.embed_matrix], trainable= False,mask_zero=True) 
        else : #fine-tuned
            self.build_embedding_matrix()
            self.embedding_layer = Embedding(self.vocab_size, self.embed_size, weights= [self.embed_matrix], trainable= True,mask_zero=True) 
        h = self.embedding_layer(x)
        
        num_layers = len(self.state_sizes)
        for i in range(num_layers):
            if (i == num_layers-1):
                layer = self.get_layer(hidden_size=self.state_sizes[i])
            else:
                layer = self.get_layer(hidden_size=self.state_sizes[i],return_sequences= True)
                
        h=layer(h)                                                     
           
        h = tf.keras.layers.Dense(dm.num_classes, activation='softmax')(h)
        self.model = tf.keras.Model(inputs=x, outputs=h)
        
        
        #Insert your code here
        
    def compile_model(self, *args, **kwargs):
        self.model.compile(*args, **kwargs)
    
    def fit(self, *args, **kwargs):
        return self.model.fit(*args, **kwargs)
    
    def evaluate(self, *args, **kwargs):
        self.model.evaluate(*args, **kwargs)       


#### <span style="color:red">**Question 4.3.2**</span> 

**Design the experiment to compare three running modes. Note that you should stick with fixed values for other attributes and only vary *run_mode*. Give your comments for the results.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [2]:
modes = ["scratch","init-only","init-fine-tune"]
for mode in modes:
    print("running mode: ",mode)

    rnn = RNN(data_manager=dm, run_mode= mode,cell_type= 'lstm', embed_size= 128, state_sizes= [128, 128])
    rnn.build()
    
    opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
    rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))


running mode:  scratch


NameError: name 'dm' is not defined

#### <span style="color:red">**Question 4.3.3**</span> 

**Run the above general RNN with at least five parameter sets and try to obtain the best performance. You can stick with the running mode *init-fine-tune* and use grid search to tune other parameters. Record your best model which will be used in the next part.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [ ]:
cell_types=['simple-rnn', 'gru', 'lstm']
network_type = ['bi-directional', 'uni-directional']
for nettype in network_type:
    for cell in cell_types:
        print("running cell_type: ",cell)
        print("running network_typ: ",nettype)
        rnn = RNN(data_manager=dm, run_mode= 'init-fine-tune', cell_type=cell , network_type = nettype ,embed_size= 128, state_sizes= [128, 128])
        rnn.build()
    
        opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
        rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

running cell_type:  simple-rnn
Epoch 1/20
7/7 [==============================] - 15s 580ms/step - loss: 1.5361 - accuracy: 0.3766 - val_loss: 1.3977 - val_accuracy: 0.4444
Epoch 2/20
7/7 [==============================] - 1s 203ms/step - loss: 1.1246 - accuracy: 0.5661 - val_loss: 1.1724 - val_accuracy: 0.4747
Epoch 3/20
7/7 [==============================] - 1s 201ms/step - loss: 0.9366 - accuracy: 0.6608 - val_loss: 1.0545 - val_accuracy: 0.5152
Epoch 4/20
7/7 [==============================] - 1s 197ms/step - loss: 0.7919 - accuracy: 0.7531 - val_loss: 0.9352 - val_accuracy: 0.5859
Epoch 5/20
7/7 [==============================] - 1s 205ms/step - loss: 0.6345 - accuracy: 0.8180 - val_loss: 0.7699 - val_accuracy: 0.6869
Epoch 6/20
7/7 [==============================] - 1s 199ms/step - loss: 0.4647 - accuracy: 0.8678 - val_loss: 0.5891 - val_accuracy: 0.7980
Epoch 7/20
7/7 [==============================] - 1s 201ms/step - loss: 0.3171 - accuracy: 0.9152 - val_loss: 0.4525 - val_accur

#Report your results here

Model 1 (run_mode ='init-fine-tune',...): accuracy = ...

......................................................

Model 1 (run_mode ='init-fine-tune',network_type = 'bi-directional',cell_type= 'simple-rnn', embed_model= 'glove-wiki-gigaword-100',embed_size= 128, state_sizes = [64, 64], data_manager = dm): accuracy = 0.9394

Model 2 (run_mode ='init-fine-tune',network_type = 'bi-directional',cell_type= 'gru', embed_model= 'glove-wiki-gigaword-100',embed_size= 128, state_sizes = [64, 64], data_manager = dm): accuracy = 0.9192

Model 3 (run_mode ='init-fine-tune',network_type = 'bi-directional',cell_type= 'lstm', embed_model= 'glove-wiki-gigaword-100',embed_size= 128, state_sizes = [64, 64], data_manager = dm): accuracy = 0.9596


Model 4 (run_mode ='init-fine-tune',network_type = 'uni-directional',cell_type= 'simple-rnn', embed_model= 'glove-wiki-gigaword-100',embed_size= 128, state_sizes = [64, 64], data_manager = dm): accuracy = 0.9394

Model 5 (run_mode ='init-fine-tune',network_type = 'uni-directional',cell_type= 'gru', embed_model= 'glove-wiki-gigaword-100',embed_size= 128, state_sizes = [64, 64], data_manager = dm): accuracy = 0.9394

Model 6 (run_mode ='init-fine-tune',network_type = 'uni-directional',cell_type= 'lstm', embed_model= 'glove-wiki-gigaword-100',embed_size= 128, state_sizes = [64, 64], data_manager = dm): accuracy = 0.9697



In [ ]:
#The run of your best model here
#Insert your code here

my_best_rnn = RNN(run_mode ='init-fine-tune',network_type = 'uni-directional',cell_type= 'lstm', embed_model= 'glove-wiki-gigaword-100',embed_size= 128, state_sizes = [64, 64], data_manager = dm)
my_best_rnn.build()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
my_best_rnn.compile_model(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
my_best_rnn.fit(dm.tf_train_set.batch(64), epochs=20, validation_data = dm.tf_valid_set.batch(64))

Epoch 1/20
7/7 [==============================] - 6s 324ms/step - loss: 1.6463 - accuracy: 0.3441 - val_loss: 1.4959 - val_accuracy: 0.3737
Epoch 2/20
7/7 [==============================] - 1s 107ms/step - loss: 1.3354 - accuracy: 0.4838 - val_loss: 1.3267 - val_accuracy: 0.4242
Epoch 3/20
7/7 [==============================] - 1s 111ms/step - loss: 1.1613 - accuracy: 0.5436 - val_loss: 1.1980 - val_accuracy: 0.4646
Epoch 4/20
7/7 [==============================] - 1s 106ms/step - loss: 1.0267 - accuracy: 0.6259 - val_loss: 1.1001 - val_accuracy: 0.5253
Epoch 5/20
7/7 [==============================] - 1s 105ms/step - loss: 0.9152 - accuracy: 0.6858 - val_loss: 1.0218 - val_accuracy: 0.5859
Epoch 6/20
7/7 [==============================] - 1s 104ms/step - loss: 0.8117 - accuracy: 0.7431 - val_loss: 0.9498 - val_accuracy: 0.6364
Epoch 7/20
7/7 [==============================] - 1s 108ms/step - loss: 0.7075 - accuracy: 0.8055 - val_loss: 0.8762 - val_accuracy: 0.6566
Epoch 8/20
7/7 [====

### <span style="color:#0b486b">4.4. Investigating the embedding vectors from the embedding matrix</span> ###

<div style="text-align: right"><span style="color:red; font-weight:bold">[Total marks for this part: 5 points]<span></div>

**As you know, the embedding matrix is a collection of embedding vectors, each is for one word. In this part, you will base on the cosine similarity of the embedding vectors for the words to find the top-k most relevant words for a given word.**

**Good embeddings should have words close in meaning near each other by some similarity metrics. The similarity metric we'll use is the `consine` distance, which is defined for two vector $\mathbf{u}$ and $\mathbf{v}$ as $\cos(\mathbf{u}, \mathbf{v})=\frac{\mathbf{u} \cdot \mathbf{v}}{\left\Vert{\mathbf{u}}\right\Vert\left\Vert{\mathbf{v}}\right\Vert}$ where $\cdot$ means dot product and $\left\Vert\cdot\right\Vert$ means the $\mathcal{L}_2$ norm.**

In [ ]:
def cosine_similarity(u,v):
    return np.dot(u,v)/(np.linalg.norm(u)*np.linalg.norm(u))

#### <span style="color:red">**Question 4.4.1**</span> 

**You are required to write the code for the function *find_most_similar(word= None, k=5, model= None)*. As its name, this function returns the top-k most relevant word for a given word based on the cosine similarity of the embedding vectors.**

<div style="text-align: right"><span style="color:red">[5 points]</span></div>

In [ ]:
def find_most_similar(word= None, k=5, model= None):
    try:
        table = model.word2vect.similar_by_word(word,k)#Insert your code here
    except: 
        print("Word is not in the dictionary!")
    return table# a table of the top-k most relevant word for a given word based on the cosine similarity of the embedding vectors.

Here is the example of the above function. As you can observe, the result makes sense which demonstrates that we obtain a good model with the meaningful embedding matrix.

In [ ]:
table = find_most_similar(word='poland', k=10, model= my_best_rnn)
for key,value in table:
        print(key,value)

hungary 0.7997257709503174
lithuania 0.7970392107963562
slovakia 0.7912642955780029
bulgaria 0.7554062008857727
germany 0.7552423477172852
romania 0.7494610548019409
warsaw 0.7206852436065674
latvia 0.7173702716827393
czech 0.7154167294502258
polish 0.6944576501846313


--- 
<div style="text-align: center"> <span style="color:black">END OF ASSIGNMENT</span> </div>

In [1]:
!jupyter nbconvert --to html *.ipynb


[NbConvertApp] WARNING | pattern '*.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=True]
--execute
    Execute t